# Proof of concept
## Get Groundstate of LiH with AdaptVqe
- taken from: https://qiskit-community.github.io/qiskit-nature/howtos/adapt_vqe.html

In [17]:
# imports
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
import numpy as np
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator

from qiskit_algorithms import AdaptVQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

import time
from datetime import datetime

## Define problem with ansatz

In [57]:
#driver = PySCFDriver(atom="H 0 0 0; H 0 0 0.735", basis="sto-3g")
#driver = PySCFDriver(atom="C 0 0 -0.6025; H 0 0 -1.6691; C 0 0 0.6025; H 0 0 1.6691", basis="sto-3g")
#driver = PySCFDriver(atom="O 0 0 0; H  0 1 0; H 0 0 1", basis="sto-3g")
driver = PySCFDriver(atom="Li 0 0 0; H 0 0 1.5", basis="sto-3g")
#driver = PySCFDriver(atom="He 0 0 0; He 0 0 0.9", basis="sto-3g")
problem = driver.run()

In [58]:
mapper = JordanWignerMapper()

In [59]:
ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
    ),
)

In [60]:
print(problem.num_spatial_orbitals, problem.num_particles, ansatz.width())

6 (2, 2) 12


In [61]:
# define vqe
vqe = VQE(Estimator(), ansatz, SLSQP())
vqe.initial_point = np.zeros(ansatz.num_parameters)

## implement TetrisAdaptVQE
- according to paper:
    Panagiotis G. Anastasiou, Yanzhu Chen, Nicholas J. Mayhall, Edwin Barnes,
    and Sophia E. Economou.
    TETRIS-ADAPT-VQE: An adaptive algorithm that yields shallower, denser
    circuit ansätze
    arXiv:2209.10562 (2022)
- according to sandboxAQ -> Tangelo implementation

In [23]:
# imports
import logging

from enum import Enum

from qiskit.quantum_info.operators.base_operator import BaseOperator
from qiskit.circuit.library import EvolvedOperatorAnsatz

from qiskit_algorithms.list_or_dict import ListOrDict
from qiskit_algorithms import AdaptVQEResult

from  qiskit_algorithms.observables_evaluator import estimate_observables

In [24]:
logger = logging.getLogger(__name__)

In [25]:
class TerminationCriterion(Enum):
    """A class enumerating the various finishing criteria."""

    CONVERGED = "Threshold converged"
    CYCLICITY = "Aborted due to a cyclic selection of evolution operators"
    MAXIMUM = "Maximum number of iterations reached"


In [26]:
class TetrisAdaptVQE(AdaptVQE):
    def compute_minimum_eigenvalue(
        self,
        operator: BaseOperator,
        aux_operators: ListOrDict[BaseOperator] | None = None,
    ) -> AdaptVQEResult:
        """Computes the minimum eigenvalue.

        Args:
            operator: Operator whose minimum eigenvalue we want to find.
            aux_operators: Additional auxiliary operators to evaluate.

        Raises:
            TypeError: If an ansatz other than :class:`~.EvolvedOperatorAnsatz` is provided.
            AlgorithmError: If all evaluated gradients lie below the convergence threshold in
                the first iteration of the algorithm.

        Returns:
            An :class:`~.AdaptVQEResult` which is a :class:`~.VQEResult` but also but also
            includes runtime information about the AdaptVQE algorithm like the number of iterations,
            termination criterion, and the final maximum gradient.
        """
        if not isinstance(self.solver.ansatz, EvolvedOperatorAnsatz):
            raise TypeError("The AdaptVQE ansatz must be of the EvolvedOperatorAnsatz type.")

        # Overwrite the solver's ansatz with the initial state
        self._tmp_ansatz = self.solver.ansatz
        self._excitation_pool = self._tmp_ansatz.operators
        self.solver.ansatz = self._tmp_ansatz.initial_state

        prev_op_indices: list[int] = []
        prev_raw_vqe_result: VQEResult | None = None
        raw_vqe_result: VQEResult | None = None
        theta: list[float] = []
        max_grad: tuple[float, dict[str, BaseOperator] | None] = (0.0, None)
        self._excitation_list = []
        history: list[complex] = []
        iteration = 0
        while self.max_iterations is None or iteration < self.max_iterations:
            iteration += 1
            logger.info("--- Iteration #%s ---", str(iteration))
            # compute gradients
            logger.debug("Computing gradients")
            cur_grads = self._compute_gradients(theta, operator)

            max_grad_index, max_grad, pool_select = self.get_grads_and_indices(cur_grads)

            ops_appended = 0 
            
            logger.info(
                "Found maximum gradient %s at index %s",
                str(np.abs(max_grad[0])),
                str(max_grad_index),
            )
            # log gradients
            if np.abs(max_grad[0]) < self.gradient_threshold:
                if iteration == 1:
                    raise AlgorithmError(
                        "All gradients have been evaluated to lie below the convergence threshold "
                        "during the first iteration of the algorithm. Try to either tighten the "
                        "convergence threshold or pick a different ansatz."
                    )
                logger.info(
                    "AdaptVQE terminated successfully with a final maximum gradient: %s",
                    str(np.abs(max_grad[0])),
                )
                termination_criterion = TerminationCriterion.CONVERGED
                break
            # store maximum gradient's index for cycle detection
            prev_op_indices.append(max_grad_index)
            # check indices of picked gradients for cycles
            if self._check_cyclicity(prev_op_indices):
                logger.info("Alternating sequence found. Finishing.")
                logger.info("Final maximum gradient: %s", str(np.abs(max_grad[0])))
                termination_criterion = TerminationCriterion.CYCLICITY
                break
            # add new excitation to self._ansatz
            logger.info(
                "Adding new operator to the ansatz: %s", str(self._excitation_pool[max_grad_index])
            )
            self._excitation_list.append(self._excitation_pool[max_grad_index])
            theta.append(0.0)
            ops_appended += 1

            # add other circuits if any
            if pool_select:
                print("something in pool")
                for elem in pool_select:
                    prev_op_indices.append(elem)
                    # check indices of picked gradients for cycles
                    if self._check_cyclicity(prev_op_indices):
                        logger.info("Alternating sequence found. Finishing.")
                        logger.info("Final maximum gradient: %s", str(np.abs(max_grad[0])))
                        termination_criterion = TerminationCriterion.CYCLICITY
                        break  
                    # add new additional excitation to self._ansatz
                    logger.info(
                        "Adding new operator to the ansatz: %s", str(self._excitation_pool[elem])
                    )
                    self._excitation_list.append(self._excitation_pool[elem])
                    theta.append(0.0)
                    ops_appended += 1
            
            # setting up the ansatz for the VQE iteration
            self._tmp_ansatz.operators = self._excitation_list
            self.solver.ansatz = self._tmp_ansatz
            self.solver.initial_point = np.asarray(theta)
            # evaluating the eigenvalue with the internal VQE
            prev_raw_vqe_result = raw_vqe_result
            raw_vqe_result = self.solver.compute_minimum_eigenvalue(operator)
            theta = raw_vqe_result.optimal_point.tolist()
            # checking convergence based on the change in eigenvalue
            if iteration > 1:
                eigenvalue_diff = np.abs(raw_vqe_result.eigenvalue - history[-1])
                if eigenvalue_diff < self.eigenvalue_threshold:
                    logger.info(
                        "AdaptVQE terminated successfully with a final change in eigenvalue: %s",
                        str(eigenvalue_diff),
                    )
                    termination_criterion = TerminationCriterion.CONVERGED
                    logger.debug(
                        "Reverting the addition of the last excitation to the ansatz since it "
                        "resulted in a change of the eigenvalue below the configured threshold."
                    )
                    for i in range(ops_appended):
                        self._excitation_list.pop()
                        theta.pop()
                    self._tmp_ansatz.operators = self._excitation_list
                    self.solver.ansatz = self._tmp_ansatz
                    self.solver.initial_point = np.asarray(theta)
                    raw_vqe_result = prev_raw_vqe_result
                    break
            # appending the computed eigenvalue to the tracking history
            history.append(raw_vqe_result.eigenvalue)
            logger.info("Current eigenvalue: %s", str(raw_vqe_result.eigenvalue))
        else:
            # reached maximum number of iterations
            termination_criterion = TerminationCriterion.MAXIMUM
            logger.info("Maximum number of iterations reached. Finishing.")
            logger.info("Final maximum gradient: %s", str(np.abs(max_grad[0])))

        result = AdaptVQEResult()
        result.combine(raw_vqe_result)
        result.num_iterations = iteration
        result.final_max_gradient = max_grad[0]
        result.termination_criterion = termination_criterion  # type: ignore[assignment]
        result.eigenvalue_history = history

        # once finished evaluate auxiliary operators if any
        if aux_operators is not None:
            aux_values = estimate_observables(
                self.solver.estimator,
                self.solver.ansatz,
                aux_operators,
                result.optimal_point,  # type: ignore[arg-type]
            )
            result.aux_operators_evaluated = aux_values  # type: ignore[assignment]

        logger.info("The final eigenvalue is: %s", str(result.eigenvalue))
        self.solver.ansatz.operators = self._excitation_pool
        return result

    def get_grads_and_indices(self, gradients):
        # pick maximum gradient
        max_grad_index, max_grad = max(  # type: ignore[assignment]
            enumerate(gradients),
            # mypy <= 1.10 needs call-overload, for 1.11 its arg-type to suppress the error
            # below then the other is seen as unused hence the additional unused-ignore
            key=lambda item: np.abs(
                item[1][0]  # type: ignore[arg-type, unused-ignore]
            ),  # type: ignore[call-overload, unused-ignore]
        )

        # Sorting the pool operators according to the gradients.
        sorted_op_indices = sorted(range(len(gradients)), key=lambda k: gradients[k][0])

        qubit_indices = set(range(self._tmp_ansatz.width()))
        op_indices_to_add = list()
        
        for i in sorted_op_indices[::-1]:
            #print(i, gradients[i][0], self.gradient_threshold)
            # If gradient is lower than the tolerance, all the remaining
            # operators have a lower gradient also. If there is no "available"
            # qubit anymore, no more operator can be added.
            if np.abs(gradients[i][0]) < self.gradient_threshold or len(qubit_indices) == 0:
                break

            op_indices = set(range(self._excitation_pool[i].num_qubits))
            #print("here", op_indices)

            # If the operator acts on a subset of "available" qubits, it can be
            # considered for this ADAPT cycle. Those qubit indices are then
            # removed from consideration for this ADAPT cycle.
            if op_indices.issubset(qubit_indices):
                qubit_indices -= op_indices
                op_indices_to_add.append(i)
        
        return sorted_op_indices[0], gradients[sorted_op_indices[0]], op_indices_to_add


## now setup AdaptVQE

In [27]:
tetris_adapt_vqe = TetrisAdaptVQE(vqe)
tetris_adapt_vqe.supports_aux_operators = lambda: True  # temporary fix

## Now get some results for TetrisAdaptVQE

In [28]:
solver = GroundStateEigensolver(mapper, tetris_adapt_vqe)

In [29]:
start = time.time()
result = solver.solve(problem)
end = time.time()
# print execution time
print('Code execution time [sec]:', end - start)
print(f"Total ground state energy = {result.total_energies[0]:.4f}")

something in pool
something in pool
something in pool
something in pool
something in pool
something in pool
something in pool
something in pool
something in pool
something in pool
something in pool
something in pool
Code execution time [sec]: 1493.7961044311523
Total ground state energy = -7.8820


## Now get some results for AdaptVQE

In [14]:
adapt_vqe = AdaptVQE(vqe)
adapt_vqe.supports_aux_operators = lambda: True  # temporary fix

In [15]:
solver = GroundStateEigensolver(mapper, adapt_vqe)

In [16]:
start = time.time()
result = solver.solve(problem)
end = time.time()
# print execution time
print('Code execution time [sec]:', end - start)
print(f"Total ground state energy = {result.total_energies[0]:.4f}")

Code execution time [sec]: 1257.5359916687012
Total ground state energy = -7.8820


## Implement StatefullAdaptVQE -> from qiskit_nature_cp2k

In [62]:
def depth_filter(param):
    inst, qubits, clbits = param
    return inst.num_qubits == 2

In [63]:
class StatefulAdaptVQE(AdaptVQE):
    """A stateful AdaptVQE variant."""

    def compute_minimum_eigenvalue(
        self, operator, aux_operators=None
    ) -> AdaptVQEResult:
        if not isinstance(self.solver.ansatz, EvolvedOperatorAnsatz) and not isinstance(self._tmp_ansatz, EvolvedOperatorAnsatz):
            raise TypeError(
                "The AdaptVQE ansatz must be of the EvolvedOperatorAnsatz type."
            )

        if self._tmp_ansatz is None:
            # Overwrite the solver's ansatz with the initial state
            self._tmp_ansatz = self.solver.ansatz
            self._excitation_pool = self._tmp_ansatz.operators
            self.solver.ansatz = self._tmp_ansatz.initial_state
            self._theta: list[float] = []
            self._excitation_list = []
            self._global_iteration = 1
            self._prev_op_indices: list[int] = []
            self._history: list[complex] = []
            self._prev_raw_vqe_result: VQEResult | None = None
        else:
            if len(self._excitation_list) == 0:
                self.solver.ansatz = self._tmp_ansatz.initial_state
            else:
                self.solver.ansatz = self._tmp_ansatz
                self.solver.initial_point = self._theta
            self._global_iteration += 1

        raw_vqe_result: VQEResult | None = None
        max_grad: tuple[complex, dict[str, Any] | None] = (0.0, None)
        iteration = 0
        while self.max_iterations is None or iteration < self.max_iterations:
            iteration += 1
            logger.info("--- Iteration #%s ---", str(iteration))
            # compute gradients
            logger.debug("Computing gradients")
            cur_grads = self._compute_gradients(self._theta, operator)
            # pick maximum gradient
            max_grad_index, max_grad = max(
                enumerate(cur_grads), key=lambda item: np.abs(item[1][0])
            )
            logger.info(
                "Found maximum gradient %s at index %s",
                str(np.abs(max_grad[0])),
                str(max_grad_index),
            )
            # log gradients
            if np.abs(max_grad[0]) < self.gradient_threshold:
                if iteration == 1 and self._global_iteration == 1:
                    logger.warning(
                        "All gradients have been evaluated to lie below the convergence threshold "
                        "during the first iteration of the algorithm. Try to either tighten the "
                        "convergence threshold or pick a different ansatz."
                    )
                    raw_vqe_result = self.solver.compute_minimum_eigenvalue(operator)
                    # store this current VQE result for the potential stateful restart later on
                    self._prev_raw_vqe_result = raw_vqe_result
                    self._theta = raw_vqe_result.optimal_point
                    termination_criterion = TerminationCriterion.CONVERGED
                    break
                logger.info(
                    "AdaptVQE terminated successfully with a final maximum gradient: %s",
                    str(np.abs(max_grad[0])),
                )
                termination_criterion = TerminationCriterion.CONVERGED
                break
            # store maximum gradient's index for cycle detection
            self._prev_op_indices.append(max_grad_index)
            # check indices of picked gradients for cycles
            if self._check_cyclicity(self._prev_op_indices):
                logger.info("Alternating sequence found. Finishing.")
                logger.info("Final maximum gradient: %s", str(np.abs(max_grad[0])))
                termination_criterion = TerminationCriterion.CYCLICITY
                break
            # add new excitation to self._ansatz
            logger.info(
                "Adding new operator to the ansatz: %s",
                str(self._excitation_pool[max_grad_index]),
            )
            self._excitation_list.append(self._excitation_pool[max_grad_index])
            self._theta.append(0.0)
            # setting up the ansatz for the VQE iteration
            self._tmp_ansatz.operators = self._excitation_list
            self.solver.ansatz = self._tmp_ansatz
            self.solver.initial_point = self._theta
            # evaluating the eigenvalue with the internal VQE
            self._prev_raw_vqe_result = raw_vqe_result
            raw_vqe_result = self.solver.compute_minimum_eigenvalue(operator)
            self._theta = raw_vqe_result.optimal_point.tolist()
            # checking convergence based on the change in eigenvalue
            if iteration > 1:
                eigenvalue_diff = np.abs(raw_vqe_result.eigenvalue - self._history[-1])
                if eigenvalue_diff < self.eigenvalue_threshold:
                    logger.info(
                        "AdaptVQE terminated successfully with a final change in eigenvalue: %s",
                        str(eigenvalue_diff),
                    )
                    termination_criterion = TerminationCriterion.CONVERGED
                    logger.debug(
                        "Reverting the addition of the last excitation to the ansatz since it "
                        "resulted in a change of the eigenvalue below the configured threshold."
                    )
                    self._excitation_list.pop()
                    self._theta.pop()
                    self._tmp_ansatz.operators = self._excitation_list
                    self.solver.ansatz = self._tmp_ansatz
                    self.solver.initial_point = self._theta
                    raw_vqe_result = self._prev_raw_vqe_result
                    break
            # appending the computed eigenvalue to the tracking history
            self._history.append(raw_vqe_result.eigenvalue)
            logger.info("Current eigenvalue: %s", str(raw_vqe_result.eigenvalue))
        else:
            # reached maximum number of iterations
            self._prev_raw_vqe_result = raw_vqe_result
            termination_criterion = TerminationCriterion.MAXIMUM
            logger.info("Maximum number of iterations reached. Finishing.")
            logger.info("Final maximum gradient: %s", str(np.abs(max_grad[0])))

        if raw_vqe_result is None:
            raw_vqe_result = self._prev_raw_vqe_result
        result = AdaptVQEResult()
        result.combine(raw_vqe_result)
        result.num_iterations = iteration
        result.final_max_gradient = max_grad[0]
        result.termination_criterion = termination_criterion
        result.eigenvalue_history = self._history

        # once finished evaluate auxiliary operators if any
        if aux_operators is not None:
            aux_values = estimate_observables(
                self.solver.estimator,
                self.solver.ansatz,
                aux_operators,
                result.optimal_point,
            )
            result.aux_operators_evaluated = aux_values

        logger.info("The final eigenvalue is: %s", str(result.eigenvalue))
        decomposed = self.solver.ansatz.decompose().decompose().decompose()
        logger.info(f"The current circuit has the following gates: %s", str(decomposed.count_ops()))
        logger.info(f"The current circuit has a depth of %s", str(decomposed.depth()))
        logger.info(f"The current circuit has a 2-qubit gate depth of %s", str(decomposed.depth(depth_filter)))
        logger.info(f"The current circuit has %s paramters", str(decomposed.num_parameters))
        return result 

## Now get some results from StatefullAdaptVQE

In [64]:
stateful_adapt_vqe = StatefulAdaptVQE(
    vqe,           
    eigenvalue_threshold=1e-4, #1e-6      
    gradient_threshold=1e-4,       
    max_iterations=20,
)
stateful_adapt_vqe.supports_aux_operators = lambda: True # temporary Fix

In [65]:
solver = GroundStateEigensolver(mapper, stateful_adapt_vqe)

In [66]:
start = time.time()
result = solver.solve(problem)
end = time.time()
# print execution time
print('Code execution time [sec]:', end - start)
print(f"Total ground state energy = {result.total_energies[0]:.4f}")

Code execution time [sec]: 272.2553725242615
Total ground state energy = -7.8802
